Для каждого значения столбца key необходимо найти:

- Длительность времени между первым и последним замером температуры.
- Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов.
- Количество запусков нагрева электродами.
- Среднее соотношение потребления активной и реактивной мощности.

По всем полученным столбцам вычислить статистики: средние, минимальные и максимальные значения, медиану и величины 25%- и 75%-квартилей.

In [1]:
!pip install pyod
!pip install xgboost
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import datetime
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from pyod.models.knn import KNN
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
warnings.filterwarnings('ignore')
%matplotlib inline

# Длительность времени между первым и последним замером температуры.

In [2]:
### Импорт данных

try:
    data_temp = pd.read_csv('/Users/Ivan/data_temp.csv', parse_dates=['Время замера'])
except:
    data_temp = pd.read_csv('https://code.s3.yandex.net/datasets/data_temp.csv')

data_temp.head(5)

,key,Время замера,Температура
0,1,2019-05-03 11:16:18,1571.0
1,1,2019-05-03 11:25:53,1604.0
2,1,2019-05-03 11:29:11,1618.0
3,1,2019-05-03 11:30:01,1601.0
4,1,2019-05-03 11:30:39,1613.0


In [3]:
start_time = data_temp.sort_values('Время замера').groupby('key').first()['Время замера']
end_time = data_temp.sort_values('Время замера').groupby('key').last()['Время замера']
full_time = end_time - start_time
full_time.name = 'full_time'
full_time.head()

key
1   0 days 00:14:21
2   0 days 00:21:45
3   0 days 00:21:40
4   0 days 00:06:28
5   0 days 00:12:42
Name: full_time, dtype: timedelta64[ns]

# Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов.

In [4]:
### Импорт данных

try:
    data_arc = pd.read_csv('/Users/Ivan/data_arc.csv', 
                     parse_dates=['Начало нагрева дугой', 'Конец нагрева дугой'])
except:
    data_arc = pd.read_csv('https://code.s3.yandex.net/datasets/data_arc.csv')

data_arc.head()

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397


In [5]:
data_arc['time'] = (data_arc['Конец нагрева дугой'] - data_arc['Начало нагрева дугой'])
data_arc.head()

,key,Начало нагрева дугой,Конец нагрева дугой,Активная мощность,Реактивная мощность,time
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084,0 days 00:03:48
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285,0 days 00:03:05
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805,0 days 00:02:52
3,1,2019-05-03 11:18:14,2019-05-03 11:24:19,1.659363,1.062669,0 days 00:06:05
4,1,2019-05-03 11:26:09,2019-05-03 11:28:37,0.692755,0.414397,0 days 00:02:28


In [6]:
arc_time = data_arc.groupby('key').agg({'time': np.sum})
arc_time.rename({'time': 'arc_time'}, axis=1, inplace=True)
arc_time.head()

,arc_time
key,
1,0 days 00:18:18
2,0 days 00:13:31
3,0 days 00:10:55
4,0 days 00:12:21
5,0 days 00:14:29


#  Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов.

In [7]:
arc_starts = data_arc.groupby('key').count()['time']
arc_starts.name = 'arc_starts'
arc_starts.head()

key
1    5
2    4
3    5
4    4
5    4
Name: arc_starts, dtype: int64

# Среднее соотношение потребления активной и реактивной мощности.

In [8]:
data_arc['power_ratio'] = (data_arc['Активная мощность'] / data_arc['Реактивная мощность'])
data_arc['power_ratio'].head()

0    1.420582
1    1.548395
2    1.492292
3    1.561504
4    1.671717
Name: power_ratio, dtype: float64

In [9]:
data_arc['power_ratio'].describe()

count    14876.000000
mean         1.360584
std          0.173451
min         -0.000693
25%          1.257090
50%          1.376088
75%          1.480588
max          1.949831
Name: power_ratio, dtype: float64

In [10]:
mean_power_ratio = data_arc.groupby('key').mean()['power_ratio']
mean_power_ratio.head()

key
1    1.538898
2    1.551834
3    1.583120
4    1.563487
5    1.480428
Name: power_ratio, dtype: float64

# По всем полученным столбцам вычислите статистики: средние, минимальные и максимальные значения, медиану и величины 25%- и 75%-квартилей.

In [11]:
data = pd.DataFrame(full_time).join([arc_time, arc_starts, mean_power_ratio])
data.head()

,full_time,arc_time,arc_starts,power_ratio
key,,,,
1,0 days 00:14:21,0 days 00:18:18,5.0,1.538898
2,0 days 00:21:45,0 days 00:13:31,4.0,1.551834
3,0 days 00:21:40,0 days 00:10:55,5.0,1.583120
4,0 days 00:06:28,0 days 00:12:21,4.0,1.563487
5,0 days 00:12:42,0 days 00:14:29,4.0,1.480428


In [12]:
data.describe()

,full_time,arc_time,arc_starts,power_ratio
count,3216,3214,3214.000000,3214.000000
mean,0 days 00:30:39.861629353,0 days 00:13:14.579029247,4.628500,1.357452
std,0 days 00:20:46.950298194,0 days 00:05:32.448622113,1.608647,0.132824
min,0 days 00:00:00,0 days 00:00:57,1.000000,0.676106
25%,0 days 00:19:37,0 days 00:09:31,4.000000,1.292892
50%,0 days 00:26:21,0 days 00:12:50,4.000000,1.370550
75%,0 days 00:37:15,0 days 00:16:23,6.000000,1.438292
max,0 days 06:32:17,0 days 01:09:49,16.000000,1.782164
